Jan 11, 2021

In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row
import pandas as pd
import numpy as np

from pyspark.sql.functions import when, udf, col, regexp_replace
from pyspark.sql.types import DoubleType,IntegerType, StringType

# 스파크 통계
import pyspark.sql.functions as F

In [2]:
sc = SparkContext( 'local' ) 
sqlCtx = SQLContext( sc )

# Join
- 데이터 프레임의 멤버함수
- **결합**함수
- 인자
    - on = **['기준 컬럼']**
        - ! 각 데이터 프레임에 공통 컬럼이 있어야함
    - how = 'inner' or 'outer' or 'left' or 'right' ...
        - default 값 : inner

# 예제1

In [3]:
emp =[('홍길동',1),('이순신',2),
      ('임꺽정',3),('김철수',3),('김철수1',5)]
dept = [('개발',1), ('연구',2),
        ('영업',3),('기획',4) ]

empA=sqlCtx.createDataFrame( emp, ['name','deptid'] )
deptB=sqlCtx.createDataFrame( dept,['deptname','deptid'] )

In [4]:
empA.show()

+-------+------+
|   name|deptid|
+-------+------+
| 홍길동|     1|
| 이순신|     2|
| 임꺽정|     3|
| 김철수|     3|
|김철수1|     5|
+-------+------+



In [5]:
deptB.show()

+--------+------+
|deptname|deptid|
+--------+------+
|    개발|     1|
|    연구|     2|
|    영업|     3|
|    기획|     4|
+--------+------+



In [16]:
# inner join
# null이 있는건 **제거**한다는 의미
empA.join( deptB, on=['deptid'] ).show() # default값은 inner

+------+------+--------+
|deptid|  name|deptname|
+------+------+--------+
|     1|홍길동|    개발|
|     3|임꺽정|    영업|
|     3|김철수|    영업|
|     2|이순신|    연구|
+------+------+--------+



In [17]:
# left join : 왼쪽이 다 나와야한다.
empA.join( deptB, on=['deptid'], how='left' ).show()

+------+-------+--------+
|deptid|   name|deptname|
+------+-------+--------+
|     5|김철수1|    null|
|     1| 홍길동|    개발|
|     3| 임꺽정|    영업|
|     3| 김철수|    영업|
|     2| 이순신|    연구|
+------+-------+--------+



In [18]:
# right join :  오른쪽이 다 나와야한다.
empA.join( deptB, on=['deptid'], how='right' ).show()

+------+------+--------+
|deptid|  name|deptname|
+------+------+--------+
|     1|홍길동|    개발|
|     3|임꺽정|    영업|
|     3|김철수|    영업|
|     2|이순신|    연구|
|     4|  null|    기획|
+------+------+--------+



In [21]:
# full join : 두쪽 다 나와야한다
empA.join( deptB, on=['deptid'], how='full' ).show()

+------+-------+--------+
|deptid|   name|deptname|
+------+-------+--------+
|     5|김철수1|    null|
|     1| 홍길동|    개발|
|     3| 임꺽정|    영업|
|     3| 김철수|    영업|
|     2| 이순신|    연구|
|     4|   null|    기획|
+------+-------+--------+



# 예제2

In [22]:
testA = [('A',1),('B',2),('C',3),('D',4)]
testB = [('E',1),('A',2),('C',3),('F',4)]
testAA = sqlCtx.createDataFrame( testA, ['name','id'] )
testBB = sqlCtx.createDataFrame( testB,['name', 'myid'] )
testAA.show()
testBB.show()

+----+---+
|name| id|
+----+---+
|   A|  1|
|   B|  2|
|   C|  3|
|   D|  4|
+----+---+

+----+----+
|name|myid|
+----+----+
|   E|   1|
|   A|   2|
|   C|   3|
|   F|   4|
+----+----+



In [29]:
# inner join : a,c만 나옴
testAA.join( testBB, on=['name'] ).show()

+----+---+----+
|name| id|myid|
+----+---+----+
|   C|  3|   3|
|   A|  1|   2|
+----+---+----+



In [31]:
# left join : testAA가 다 나와야한다
testAA.join( testBB, on=['name'], how='full' ).show()

+----+----+----+
|name|  id|myid|
+----+----+----+
|   F|null|   4|
|   E|null|   1|
|   B|   2|null|
|   D|   4|null|
|   C|   3|   3|
|   A|   1|   2|
+----+----+----+



In [32]:
# right join
testAA.join( testBB, on=['name'], how='right' ).show()

+----+----+----+
|name|  id|myid|
+----+----+----+
|   F|null|   4|
|   E|null|   1|
|   C|   3|   3|
|   A|   1|   2|
+----+----+----+



In [36]:
# full join
dd = testAA.join( testBB, on=['name'], how='full' )
dd.show()

+----+----+----+
|name|  id|myid|
+----+----+----+
|   F|null|   4|
|   E|null|   1|
|   B|   2|null|
|   D|   4|null|
|   C|   3|   3|
|   A|   1|   2|
+----+----+----+



# Null 처리

## (1) null 제거
- dropna()
    - **subset = ['drop을 수행할 컬럼']**

In [39]:
dd.dropna().show()  # inner join과 같은 결과

+----+---+----+
|name| id|myid|
+----+---+----+
|   C|  3|   3|
|   A|  1|   2|
+----+---+----+



In [38]:
dd.dropna( subset=['id'] ).show() 

+----+---+----+
|name| id|myid|
+----+---+----+
|   B|  2|null|
|   D|  4|null|
|   C|  3|   3|
|   A|  1|   2|
+----+---+----+



## (2) null 채우기
- fillna( **{  }** )
    - **컬럼명**과 **null**을 대체할 값

In [42]:
dd.fillna( {'id':100, 'myid':200 } ).show()

+----+---+----+
|name| id|myid|
+----+---+----+
|   F|100|   4|
|   E|100|   1|
|   B|  2| 200|
|   D|  4| 200|
|   C|  3|   3|
|   A|  1|   2|
+----+---+----+



# Union
- 특정 컬럼기준으로 결합 X
- **행단위로 합침**

In [43]:
testAA.unionAll( testBB ).show()

+----+---+
|name| id|
+----+---+
|   A|  1|
|   B|  2|
|   C|  3|
|   D|  4|
|   E|  1|
|   A|  2|
|   C|  3|
|   F|  4|
+----+---+

